In [1]:
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver # 브라우저를 조작하는 모듈
from selenium.webdriver.chrome.service import Service # 크롬 드라이버의 시작과 중지를 담당하는 클래스
from selenium.webdriver.common.by import By # 엘리먼트를 어떤 방식으로 선택할 지에 대한 상수
from webdriver_manager.chrome import ChromeDriverManager # 크롬 드라이버를 관리
import pandas as pd
from tqdm.auto import tqdm
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys

In [2]:
def all_review(soup): # 모든 리뷰 갯수 찾아오는 함수
    review = soup.find('span', attrs={'class':'_3HJHJjSrNK'}).text # soup에서 찾아라. <span class="_3HJHJjSrNK">20,813</span>를 여기서 text(20,813)만 가져와라
    review = int(review.replace(',', '')) # ,날리고 정수만 남겨서 저장
    return review

In [3]:
def review_per_page(soup): # 하나의 페이지에 있는 리뷰(20개) 찾아오는 함수
    content= soup.find_all('div', attrs={'class':'YEtwtZFLDz'}) # find_all을 씀으로써 순서대로 20개의 댓글 찾아옴
    return len(content) # 갯수 반환(마지막 페이지에서만 갯수가 다를 수 있음!)

In [4]:
def next_click(driver): 
    driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq').click()

In [5]:
def review_click(driver): # 리뷰 클릭 동작하는 함수
    selector = "a[href='#'][class='_11xjFby3Le N=a:tab.review']" # 리뷰 버튼 위치를 selector로 지정
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector)))
    button.click()

In [6]:
def data_collect(soup, i):
    # 댓글 본문 수집
    content= soup.find_all('div', attrs={'class':'YEtwtZFLDz'})[i].text
    content = content.replace('\n', ' ')
    
    # 평점 수집
    score = soup.select('div._1YShY6EQ56 > div > div > div > em._15NU42F3kT')[i].text
    score = int(score)
    
    # 날짜 수집
    date = soup.select('div._2FmJXrTVEX > span._3QDEeS6NLn')[i].text
    
    # 옵션 수집
    option = soup.select('div._14FigHP3K8')[i] # 옵션 + 평가(=특징)까지 같이 포함되어 있는 경로
    # extract이 pop함수와 비슷하다고 생각하시면 편합니다. 하위 태그인 dl을 반환과 동시에 삭제하는 함수입니다.
    try:
        satis = option.find('dl').extract()
    except:
        pass
    option = option.text
    
    # 평가 수집
    satis_list = []
    try:
        for j in range(len(satis.select('dt'))): # satis에 들어간 태그 안에 dt(=평가(헤어,세정력,두피 등))가 몇 개 있는지
            text = satis.select('dt')[j].text +'_'+ satis.select('dd')[j].text
            satis_list.append(text)
    except:
        text = ''
        satis_list.append(text)
    data = [content, score, date, option, satis_list]
    return data

In [7]:
url_list = ['https://smartstore.naver.com/beautyflex/products/5381572112?NaPm=ct%3Dlm9xlkrc%7Cci%3Dda27006983e50fd5adccc65fa291e87b122749a5%7Ctr%3Dslsl%7Csn%3D1133061%7Chk%3D494696a747bde4e31bbc9badc652342d17560924']

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') # 전체 화면
options.add_argument("--incognito") # 시크릿 모드
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

for url in tqdm(url_list, desc="outer", position=0):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href='#'][class='_11xjFby3Le N=a:tab.review']")))
    # WebDriverWait를 통해 driver를 최대 10초동안 기다린다.
    # 언제까지? -> ID가 "a[href='#'][class='_11xjFby3Le N=a:tab.review']" 인 엘리먼트가 나올 때까지
    review_click(driver) # 리뷰 버튼 클릭(이 버튼을 눌러야 리뷰들이 로딩됨)
    driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.PAGE_DOWN)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.YEtwtZFLDz")))    
    # WebDriverWait를 통해 driver를 최대 10초동안 기다린다.    
    # 언제까지? -> ID가 "div.YEtwtZFLDz" 인 엘리먼트가 나올때까지(해당 엘리먼트는 '댓글'의 위치. 즉, 한 페이지에 20개의 동일한 엘리먼트가 존재)
    html = driver.page_source # 위의 작업이 수행된 후 해당 접속 사이트 url 정보 가져옴(html로 변수 지정)
    soup = BeautifulSoup(html, 'lxml') # html(변수)를 통해 가져온 HTML 문서를 lxml 파서를 통해서 BeautifulSoup 객체로 만들어줌
    
    review = all_review(soup)
    
    page_num = (review // 20) + 1
    result = []
    # 1페이지 수집
    for i in range(review_per_page(soup)):
        data = data_collect(soup, i)
        result.append(data)

    # 2페이지 부터 수집
    for _ in tqdm(range(page_num), desc='inner', position=1):
        try:
            next_click(driver)
        except:
            break
        time.sleep(0.1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        
        for j in range(review_per_page(soup)):
            data = data_collect(soup, j)
            result.append(data)
                
            # time.sleep(0.3)

    results_df = pd.DataFrame(result)
    results_df.columns = ['content', 'score', 'date', 'option', 'satis_list']

outer:   0%|          | 0/1 [00:00<?, ?it/s]

inner:   0%|          | 0/270 [00:00<?, ?it/s]

In [9]:
results_df

,content,score,date,option,satis_list
0,작은용량이 필요했는데 딱 좋습니다,5,23.08.29.,상품선택: 할인행사) 올뉴플러스TS샴푸 100g x 5개,"[두피자극_보통이에요, 탈모케어_아주 만족해요, 세정력_아주 만족해요]"
1,써보지는 않았지만 좋다고 해서 주문했어요,5,23.08.29.,상품선택: [탈모케어] 골드플러스TS샴푸500g,"[세정력_보통이에요, 두피자극_보통이에요, 탈모케어_잘 모르겠어요]"
2,아직 사용전이지만 사용후기가 너무 좋아서 사용해 보려 합니다 한달후 사용 후기 올리...,5,23.08.19.,상품선택: [탈모케어] 골드플러스TS샴푸500g,"[헤어_건성가는 모발탈모, 두피자극_순해요, 탈모케어_아주 만족해요, 세정력_아주 ..."
3,재구매빠른 배송 감사합니다 꾸준히 사용하고 있어요,5,23.09.07.,상품선택: [탈모케어] 골드플러스TS샴푸500g,"[두피자극_순해요, 탈모케어_아주 만족해요, 세정력_아주 만족해요]"
4,빠른배송및 품질에대해해서 만족합니다,5,23.08.13.,상품선택: [탈모케어] 올뉴플러스TS샴푸 500g,"[헤어_건성탈모, 두피자극_순해요, 탈모케어_아주 만족해요, 세정력_아주 만족해요]"
...,...,...,...,...,...
5378,항상 쓰는거라 만족하는 제품이에요,5,21.02.21.,상품선택: 골드플러스TS샴푸500g (탈모케어),"[두피자극_순해요, 탈모케어_아주 만족해요, 세정력_아주 만족해요]"
5379,이 제품만써요! 배송도빠르고좋네요,5,21.02.17.,상품선택: 뉴프리미엄 TS 샴푸 500g (탈모케어),"[세정력_보통이에요, 두피자극_순해요, 탈모케어_잘 모르겠어요]"
5380,탐모샴푸.비듬샴푸 굿굿,5,21.02.25.,상품선택: 할인행사) TS 비디샴푸 100g x 5개,"[두피자극_순해요, 탈모케어_아주 만족해요, 세정력_아주 만족해요]"
5381,굿굿조아요ㅎㅎㅎ이야조아,5,21.02.20.,상품선택: 뉴프리미엄 TS 샴푸 500g (탈모케어),"[세정력_보통이에요, 두피자극_보통이에요, 탈모케어_잘 모르겠어요]"


In [10]:
results_df.to_csv('C:/Users/csjin/Desktop/crawling/data/TS샴푸.csv', index=False, encoding='utf-8-sig')